## Data Extraction and Prerocessing


In [ ]:
#Genel komutlar

import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
#RMSE ile tahmin hatalarımı belirlemek için sqrt çağırdım.(evaluate forecast)
from math import sqrt
from sklearn.metrics import mean_squared_error
#Verisetini ayıklamak için çağırdım
from numpy import split
from numpy import array



path = "../input/solar-radiation-dataset/2017_2019.csv"
df = pd.read_csv(path)


print(df)

Sütun bilgisi almak üzere info komutu kullanılır, eksik ve boş sütunlar düzenlenir.
Veriseti işlenecek hale getirilir.

In [ ]:
df.info()



*   Ay ,yıl ve saat değerleri işlenebilecek formatta.Bu hazırlama işlemini kolaylaştıracaktır.
*  Verisetinde tek yapılması gereken Değersiz sütunu çıkarmak olacaktır.





In [ ]:
df.tail(5)

In [ ]:
df=df.drop(['Unnamed: 18'], axis = 1) 

Date -Time Ayarlama 

In [ ]:
cols = ["Year","Month","Day","Hour","Minute"]
df["date_time"] = df[cols].apply(lambda row: "-".join(row.values.astype(str)), axis=1)


In [ ]:
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d-%H-%M')

Diğerlerini silme

In [ ]:
df=df.drop(["Year","Month","Day","Hour","Minute","Clearsky DHI","Clearsky DNI","Clearsky GHI","DHI" ,"DNI" ],axis = 1)

Zamanı İndex haline getirmek

In [ ]:
#df.reset_index() # Sıfırlamak içn
#df.assign(State=df.index.get_level_values('State'))#İndexi column yapmak için
df=df.set_index('date_time') #Columnu index yapmak için

In [ ]:
df.index.max()

Visualisation

In [ ]:
import matplotlib.pyplot as plt

large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')

In [ ]:
plt.plot(df["Solar Zenith Angle"])

In [ ]:
plt.plot(df["GHI"])

Haftalık/Aylık/ Yıllık Kesitler

In [ ]:
df['GHI']["2017-01-01":"2017-01-15"].plot(figsize=(16,4))

Aylık Ortalama değerler

In [ ]:
df["Temperature"].plot(figsize=(16,5))
df.rolling(window=60).mean()["Temperature"].plot();

In [ ]:
from matplotlib import dates

#VeriSetini Hazırlamak











1.  Input ve Output değerlerini belirlemek
2.  Train / Test setini  ayırmak
1.  Verisetini Ölçeklendirmek
2.   


 





Kullanılacak algoritmalarda en iyi parametreleri bulmak için optuma modülü yüklendi

In [ ]:
!pip install --quiet optuna

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import re
import optuna 
import xgboost as xgb

from sklearn.metrics import r2_score  


1.Output değeri olarak GHI seçiliyor. Global Hourly Irradiance ,Direct ve Diffuse Radiance ın toplanımda oluşturulan radyasyon değeridir.

In [ ]:
y = df["GHI"].copy()
X = df.drop("GHI",axis=1).copy()

2.Ölçeklendirme ile tüm değerlerin birbirleriyle etkileşimini dengelemiş oluruz.

In [ ]:
scaler = StandardScaler()

X=scaler.fit_transform(X)

In [ ]:
pd.DataFrame(X)

Train Test Split
Verisetini ayrımanın en iyi yolu test train ve validation olarak 3 sete 
ayırmaktır.
*   Train setinde model eğitilir .
*   Validation sette modeller karşılaştırılabilir.
*   Test sette de sonucumuz bulunur.



Önce 70/30 olarak ayıralım , veriseti büyüdükçe  train seti büyüterek değişiriz.

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, train_size=0.8,random_state=100)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train, train_size=0.8,random_state=200)

Ayrılan Setlerin büyüklükleri 

In [ ]:
print(X_train.shape[0])
print(X_val.shape[0])
print(X_test.shape[0])

Kullanacağımız XGB algoritması için Dmatrix fonksiyonuyla optimize hale getirelim.

In [ ]:
dtrain =xgb.DMatrix(X_train,label=y_train)
dval =xgb.DMatrix(X_val,label=y_val)
dtest =xgb.DMatrix(X_test,label=y_test)

# Modelin Öğrenmesi Ve Optimizasyon 

XGB nin kullandığı parametreleri optimize etmek için birfonsiyon yazacağız . 
Öğrenme hızı , derinliği ve regülarizasyonlar XGB nin optimize edilmesi gereken parametreleridir.


##Hiperparametre Optimizasyonu

In [ ]:
def get_model_rmse(params):
  model =xgb.train(params,dtrain,num_boost_round=100,evals=[(dval,"eval")],early_stopping_rounds=10,verbose_eval=0)
  results = model.eval(dval)
  rmse = np.float(re.search(r'[\d.]+$', results).group(0))
  return rmse



Deneme

In [ ]:
def objective(trial):
  learning_rate=trial.suggest_loguniform("learning_rate",0.001,1) #küçük değerlerde rastgele arama yapması için loguniformla yazılmıştır.
  max_depth=trial.suggest_int("max_depth",6,10)
  l1_reg=trial.suggest_loguniform("l1_reg",0.000001,0.01)
  l2_reg=trial.suggest_loguniform("l2_reg",0.000001,0.01)

  params = {"learning_rate":learning_rate,"max_depth":max_depth,"l1_reg":l1_reg,"l2_reg":l2_reg}
  return get_model_rmse(params)

In [ ]:
hiperparametre_arama=optuna.create_study()
hiperparametre_arama.optimize(objective,n_trials=100,show_progress_bar=True)

In [ ]:
best_params = hiperparametre_arama.best_params
best_params

# Model Eğitimi

In [ ]:
model=xgb.train(best_params,dtrain,num_boost_round=10000,evals=[(dval,"eval")],early_stopping_rounds=15)

In [ ]:
y_true=np.array(y_test,dtype=np.float)
y_pred=np.array(model.predict(dtest),dtype=np.float)